In [ ]:
!gdown --id 1Y8EOFLIRCcKpe_e0pO03yCAosTRjRMtC

Downloading...
From: https://drive.google.com/uc?id=1Y8EOFLIRCcKpe_e0pO03yCAosTRjRMtC
To: /content/UTKFace.zip
347MB [00:04, 75.8MB/s]


In [ ]:
!unzip -q /content/UTKFace.zip -d data

In [ ]:
# To download checkpoints, Keras models, TFLite models
from google.colab import files
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime

In [ ]:
n = len(os.listdir('/content/data/UTKFace'))
n

23708

In [ ]:
# Image size for our model.
MODEL_INPUT_IMAGE_SIZE = [ 128 , 128 ]

# Fraction of the dataset to be used for testing.
TRAIN_TEST_SPLIT = 0.3

# Number of samples to take from dataset
NUM_SAMPLES = 20000

# Trick to one-hot encode the label.
y1 = tf.constant( [ 1. , 0. ] , dtype='float32' ) 
y2 = tf.constant( [ 0. , 1. ] , dtype='float32' ) 

# This method will be mapped for each filename in `list_ds`. 
def parse_image( filename ):

    # Read the image from the filename and resize it.
    image_raw = tf.io.read_file( filename )
    image = tf.image.decode_jpeg( image_raw , channels=3 ) 
    # image = tf.image.resize( image , MODEL_INPUT_IMAGE_SIZE ) / 255
    image = tf.cast(image, tf.float32)
    image = tf.image.resize( image , MODEL_INPUT_IMAGE_SIZE )

    # Split the filename to get the age and the gender. Convert the age ( str ) and the gender ( str ) to dtype float32.
    parts = tf.strings.split( tf.strings.split( filename , '/' )[ 2 ] , '_' )

    # One-hot encode the label
    gender = tf.strings.to_number( parts[ 1 ] )
    gender_onehot = ( gender * y2 ) + ( ( 1 - gender ) * y1 )

    return image , gender_onehot

# List all the image files in the given directory.
list_ds = tf.data.Dataset.list_files( 'data/UTKFace/*' , shuffle=True )
# Map `parse_image` method to all filenames.
dataset = list_ds.map( parse_image , num_parallel_calls=tf.data.AUTOTUNE )
dataset = dataset.take( NUM_SAMPLES )

In [ ]:
# Create train and test splits of the dataset.
num_examples_in_test_ds = int(dataset.cardinality().numpy() * TRAIN_TEST_SPLIT)

test_ds = dataset.take(num_examples_in_test_ds)
train_ds = dataset.skip(num_examples_in_test_ds)

print('Num examples in train ds {}'.format( train_ds.cardinality()))
print('Num examples in test ds {}'.format( test_ds.cardinality()))

Num examples in train ds 14000
Num examples in test ds 6000


In [ ]:
BATCH_SIZE = 128
IMG_SIZE = (128, 128)
IMG_SHAPE = IMG_SIZE + (3,)

train_ds = train_ds.batch( BATCH_SIZE )
test_ds = test_ds.batch( BATCH_SIZE )

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.mobilenet.MobileNet(input_shape=IMG_SHAPE,
                                              include_top=False,
                                              weights="imagenet"
                                              )

base_model.trainable = False

image_batch, label_batch = next(iter(train_ds))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

base_model.summary()

(128, 4, 4, 1024)
Model: "mobilenet_1.00_128"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 64, 64, 32)        128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (

In [ ]:
inputs = tf.keras.Input(shape=IMG_SHAPE)
x = tf.keras.applications.mobilenet.preprocess_input(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
# outputs = tf.keras.layers.Dense(1)(x)
outputs = tf.keras.layers.Dense( 2 , activation='softmax' )( x )
model = tf.keras.Model(inputs, outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
tf.math.truediv_1 (TFOpLambd (None, 128, 128, 3)       0         
_________________________________________________________________
tf.math.subtract_1 (TFOpLamb (None, 128, 128, 3)       0         
_________________________________________________________________
mobilenet_1.00_128 (Function (None, 4, 4, 1024)        3228864   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2050

In [ ]:
# Init ModelCheckpoint callback
save_dir_ = 'mobilenetv1_gender'  
save_dir = save_dir_ + '/{epoch:02d}-{val_accuracy:.2f}.h5'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint( 
    save_dir , 
    save_best_only=True , 
    monitor='val_accuracy' , 
    mode='max', 
)

tb_log_name = 'mobilenetv1_gender'
# Init TensorBoard Callback
logdir = os.path.join( "tb_logs" , tb_log_name )
tensorboard_callback = tf.keras.callbacks.TensorBoard( logdir )

# Init Early Stopping callback
early_stopping_callback = tf.keras.callbacks.EarlyStopping( monitor='val_accuracy' , patience=5 )

In [ ]:
# warm up

num_epochs = 1
learning_rate = 0.001

model.compile( 
    loss=tf.keras.losses.categorical_crossentropy ,
    optimizer = tf.keras.optimizers.Adam( learning_rate ) , 
    metrics=[ 'accuracy' ]
)

model.fit( 
    train_ds, 
    epochs=num_epochs,  
    validation_data=test_ds 
    # callbacks=[ checkpoint_callback , tensorboard_callback , early_stopping_callback ]
)

110/110 [==============================] - 26s 193ms/step - loss: 0.5390 - accuracy: 0.7434 - val_loss: 0.3672 - val_accuracy: 0.8388


In [ ]:
# fine-tuning
num_epochs = 30
learning_rate = 0.0001
base_model.trainable = True # unfreeze the mobilenet backbone

model.compile( 
    loss=tf.keras.losses.categorical_crossentropy ,
    optimizer = tf.keras.optimizers.Adam( learning_rate ) , 
    metrics=[ 'accuracy' ]
)

model.fit( 
    train_ds, 
    epochs=num_epochs,  
    validation_data=test_ds,
    callbacks=[ checkpoint_callback , tensorboard_callback , early_stopping_callback ]
)

Epoch 1/30
110/110 [==============================] - 47s 384ms/step - loss: 0.4401 - accuracy: 0.8161 - val_loss: 0.3105 - val_accuracy: 0.8610
Epoch 2/30
110/110 [==============================] - 45s 376ms/step - loss: 0.2692 - accuracy: 0.8868 - val_loss: 0.2331 - val_accuracy: 0.9080
Epoch 3/30
110/110 [==============================] - 45s 376ms/step - loss: 0.2342 - accuracy: 0.9018 - val_loss: 0.2043 - val_accuracy: 0.9202
Epoch 4/30
110/110 [==============================] - 45s 382ms/step - loss: 0.1996 - accuracy: 0.9196 - val_loss: 0.1823 - val_accuracy: 0.9253
Epoch 5/30
110/110 [==============================] - 45s 378ms/step - loss: 0.1919 - accuracy: 0.9186 - val_loss: 0.1499 - val_accuracy: 0.9415
Epoch 6/30
110/110 [==============================] - 45s 380ms/step - loss: 0.1845 - accuracy: 0.9280 - val_loss: 0.1424 - val_accuracy: 0.9437
Epoch 7/30
110/110 [==============================] - 45s 380ms/step - loss: 0.1644 - accuracy: 0.9342 - val_loss: 0.1327 - val_ac

In [ ]:
!pwd

/content


In [ ]:
!ls

data  drive  mobilenetv1_gender  sample_data  tb_logs  UTKFace.zip


In [ ]:
!cp -R "/content/mobilenetv1_gender" "/content/drive/MyDrive/"

In [ ]:
!cp -R "/content/tb_logs" "/content/drive/MyDrive/"